# Segmentation of large structure (using mix of all channels) and red puncta

This notebook loads the provided RGB image from GitHub, splits channels, and performs:
- a) segmentation of the large structure from the mix (mean) of all channels (requested change)
- b) segmentation of small puncta from the red channel

Both binary masks are hole-filled. Outlines are overlaid on the original image with different colors and saved to disk. Intermediate results are also saved for inspection.

## Setup: imports and output folder
We use numpy, scikit-image, scipy, matplotlib, and the two napari-plugins as requested.

In [1]:
import os
import numpy as np
from skimage.io import imread, imsave
from skimage import img_as_ubyte
from skimage.filters import threshold_otsu
from skimage.morphology import binary_closing, disk, remove_small_objects
from skimage.segmentation import find_boundaries
from scipy.ndimage import binary_fill_holes
import matplotlib.pyplot as plt

import napari_segment_blobs_and_things_with_membranes as nsbatwm
import napari_simpleitk_image_processing as nsitk

out_dir = "results_issue_314"
os.makedirs(out_dir, exist_ok=True)

/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_tools_menu/__init__.py:10: UserWarning: Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.
  warnings.warn("Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.")
/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_skimage_regionprops/_table.py:7: UserWarning: No Qt bindings could be found
  warnings.warn(str(e))


## Load the image from GitHub and save a local copy
Note: The image is read directly from the GitHub user-attachments URL. We save an 8-bit RGB PNG to the results folder for reference.

In [2]:
url = "https://github.com/user-attachments/assets/a1501925-24ca-48ed-a222-b5d8a4bb0ea4"
rgb = imread(url)
rgb8 = rgb if rgb.dtype == np.uint8 else img_as_ubyte(rgb)
imsave(os.path.join(out_dir, "input.png"), rgb8)

## Split channels and save them
We extract red, green, and blue channels. The later steps use:
- mix (mean of R,G,B) for the large structure
- red for puncta segmentation

In [3]:
red = rgb8[..., 0]
green = rgb8[..., 1]
blue = rgb8[..., 2]

imsave(os.path.join(out_dir, "channel_red.png"), red)
imsave(os.path.join(out_dir, "channel_green.png"), green)
imsave(os.path.join(out_dir, "channel_blue.png"), blue)

## a) Segment the large structure from the mix of all channels
Workflow: mean of channels → intensity rescale → Gaussian blur → Otsu threshold → hole fill → closing. We save intermediate images and the final binary mask.

In [4]:
# Mix of channels (mean)
mix = rgb8.mean(axis=2).astype(np.float32)

# Rescale intensity to 0..255 using napari-simpleitk-image-processing
mix_rescaled = nsitk.rescale_intensity(mix, output_minimum=0, output_maximum=255).astype(np.uint8)
imsave(os.path.join(out_dir, "mix.png"), mix_rescaled)

# Denoise using napari-segment-blobs-and-things-with-membranes
mix_blur = nsbatwm.gaussian_blur(mix_rescaled, sigma=2)

# Normalize blurred image for saving
mb_min, mb_ptp = float(np.min(mix_blur)), float(np.ptp(mix_blur))
mix_blur_norm = (mix_blur - mb_min) / (mb_ptp + 1e-9)
imsave(os.path.join(out_dir, "mix_blur.png"), img_as_ubyte(np.clip(mix_blur_norm, 0, 1)))

# Global threshold (Otsu)
t_mix = threshold_otsu(mix_blur)
big_bin = mix_blur > t_mix

# Fill holes and smooth boundaries
big_filled = binary_fill_holes(big_bin)
big_filled = binary_closing(big_filled, footprint=disk(3))

imsave(os.path.join(out_dir, "big_structure_mask.png"), (big_filled.astype(np.uint8) * 255))

## b) Segment small puncta in the red channel
Workflow: white top-hat to enhance spots → Otsu threshold → remove small objects → hole fill. We save the enhanced image and the final binary mask of puncta.

In [5]:
# Enhance bright spots on dark background
red_tophat = nsbatwm.white_tophat(red, radius=3)

# Save enhanced red channel for inspection
rt_min, rt_ptp = float(np.min(red_tophat)), float(np.ptp(red_tophat))
red_tophat_norm = (red_tophat - rt_min) / (rt_ptp + 1e-9)
imsave(os.path.join(out_dir, "red_tophat.png"), img_as_ubyte(np.clip(red_tophat_norm, 0, 1)))

# Threshold
t_red = threshold_otsu(red_tophat)
puncta_bin = red_tophat > t_red

# Remove tiny objects and fill small holes
puncta_bin = remove_small_objects(puncta_bin, min_size=9)
puncta_filled = binary_fill_holes(puncta_bin)

imsave(os.path.join(out_dir, "puncta_mask.png"), (puncta_filled.astype(np.uint8) * 255))

## Create outlines and blend over the original image
We compute outlines for a) and b) and overlay them on the original RGB image. The large structure outline is shown in green, the puncta outline in magenta. We save both the composite image and a matplotlib figure.

In [6]:
# Compute boundaries
bnd_big = find_boundaries(big_filled, mode="inner")
bnd_puncta = find_boundaries(puncta_filled, mode="inner")

# Blend outlines over the original
overlay = rgb8.astype(np.float32) / 255.0
overlay[bnd_big] = [0.0, 1.0, 0.0]      # green for large structure
overlay[bnd_puncta] = [1.0, 0.0, 1.0]   # magenta for puncta
overlay_uint8 = img_as_ubyte(np.clip(overlay, 0, 1))

# Save overlay image and a figure
imsave(os.path.join(out_dir, "overlay_outlines.png"), overlay_uint8)

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(overlay_uint8)
ax.set_title("Original with outlines: big (green) and puncta (magenta)")
ax.axis("off")
plt.tight_layout()
plt.savefig(os.path.join(out_dir, "overlay_figure.png"), dpi=200, bbox_inches="tight")
plt.close(fig)